In [16]:
from py2neo import Graph

In [17]:
graph = Graph(auth=('neo4j', 'shekhor'))

In [135]:
phrase = "sump pump on switch"

Instead of returning all cliques of a given size,
group all cliques by commonUrl,
and return the largest group.

From that group, select the first cluster.

That way, in the next iteration, we have build on the clique that has the largest potential for expansion.

In [19]:
def keywords_up_to(n):
    return ', '.join(['k'+str(i) for i in range(1,n+1)])

def relationships_up_to(n):
    return ', '.join(['r'+str(i) for i in range(1,n+1)])

def all_vars(n):
    return ', '.join([keywords_up_to(n+1), relationships_up_to(n)])

def uncollect(n):
    vars = all_vars(n)
    return ', '.join([f'group[{i}] as {var}' for i,var in enumerate(vars.split(', '))])

def construct_greedy_query(phrase, min_n_keywords = 3, n_common_urls = 4):
    """
    Iteratively build cliques of min_n_keywords where all edges have more than n_common_urls
    """
    # Find keyword
    match_phrase = 'MATCH (k1:Keyword {{keyword: "{phrase}"}})-[r1:ASSOCIATED_WITH]-(k2:Keyword)\n'\
                   'WHERE size(r1.urls) > {n_common_urls}\n'\
                   'WITH *\n'\
                    .format(phrase=phrase, n_common_urls=n_common_urls)
    find_next_keyword = ''
    for n in range(2,min_n_keywords):
        # Find k_n+1: k_n's neighbors that share at least n_common_urls urls with k_1, k_2, ..., k_n
        find_next_keyword += '\n'
        find_next_keyword += 'MATCH (k{n})-[r{n}:ASSOCIATED_WITH]-(k{nplus1})\n'\
                             'WHERE NOT k{nplus1} in [{keywords_up_to_n}]\n'\
                             'WITH *,reduce(intersect = r1.urls, r IN [{relationships_up_to_n}] | apoc.coll.intersection(intersect, r.urls)) AS commonUrls\n'\
                             'WHERE size(commonUrls) > 4\n'\
                             .format(n=n,nplus1=n+1,keywords_up_to_n=keywords_up_to(n),relationships_up_to_n=relationships_up_to(n),n_common_urls=n_common_urls)
        # Find the biggest group of clusters that share the same urls and select the top cluster
        select_best_clique = 'WITH commonUrls, collect([{all_vars}]) as groups ORDER BY size(groups) DESC LIMIT 1\n'\
                             'UNWIND groups as group\n'\
                             'WITH {uncollect} LIMIT 1\n'\
                             .format(all_vars=all_vars(n), uncollect=uncollect(n))
        find_next_keyword += select_best_clique
    return_keywords = ', '.join(['k'+str(i) for i in range(1,min_n_keywords+1)])
    # return the keywords
    return_query = 'RETURN [k in [{return_keywords}] | k.keyword] AS group\n'\
                    .format(return_keywords=return_keywords)
    return match_phrase + find_next_keyword + return_query

In [155]:
def find_largest_keyword_cluster(phrase, min_n_keywords = 3, n_common_urls = 4, range_min = 15, range_max = 30):
    # run binary search
    # start with pivot = 15
    def run_query(n_keywords):
        return graph.run(construct_greedy_query(phrase, min_n_keywords = n_keywords, n_common_urls = n_common_urls)).to_table()
    
    while range_max - range_min > 2:
        query_lo = run_query(range_min)
        query_hi = run_query(range_max)
        if query_lo and query_hi:
            # set range to query_hi, query_hi*2
            range_min = range_max
            range_max = range_max*2
        elif not query_lo and not query_hi:
            # set range to query_lo//2, query_lo
            range_max = range_min
            range_min = range_min//2
        else:
            pivot = (range_min + range_max)//2
            query = run_query(pivot)
            if query:
                range_min = pivot
            else:
                range_max = pivot
    results = run_query((range_min + range_max)//2)    
    if results:
        return results
    else:
        return run_query(range_min)

In [156]:
foo = find_largest_keyword_cluster(phrase)

15
30
22
30
22
30
26
30
26
30
26
28
27


In [157]:
foo

group
"['sump pump on switch', 'turn sump pump off in winter', 'sump pump cover install', 'watchdog sump pump float switch', '20 year old sump pump', 'cost to replace sump pump in basement', 'pit boss sump pump', 'sump pump problem solving', 'sump pump preventative maintenance', 'my sump pump smells like sewage', 'sump pump cooling oil', 'what does it cost for a sump pump', 'difference between sump pump and well pump', 'zoeller m53 sump pump float adjustment', 'rain barrel sump pump discharge', 'small sump pump with internal float', 'diy sump pump install crawl space', 'back up sump pump cost', 'sump pump tuyauterie', 'can you drain a shower into a sump pump', 'my sump pump is making a loud noise', 'sump pump backup systems water pressure', 'sump pump power pack', 'why does sump pump need a vent hole', 'how to drain water from sump pump', 'sump pump lid gasket']"


In [152]:
(30+26)//2

28

In [145]:
%%time
foo = graph.run(construct_greedy_query(phrase, min_n_keywords = 26)).to_table()

CPU times: user 0 ns, sys: 3.48 ms, total: 3.48 ms
Wall time: 77.5 ms


In [146]:
foo

group
"['sump pump on switch', 'turn sump pump off in winter', 'sump pump cover install', 'watchdog sump pump float switch', '20 year old sump pump', 'cost to replace sump pump in basement', 'pit boss sump pump', 'sump pump problem solving', 'sump pump preventative maintenance', 'my sump pump smells like sewage', 'sump pump cooling oil', 'what does it cost for a sump pump', 'difference between sump pump and well pump', 'zoeller m53 sump pump float adjustment', 'rain barrel sump pump discharge', 'small sump pump with internal float', 'diy sump pump install crawl space', 'back up sump pump cost', 'sump pump tuyauterie', 'can you drain a shower into a sump pump', 'my sump pump is making a loud noise', 'sump pump backup systems water pressure', 'sump pump power pack', 'why does sump pump need a vent hole', 'how to drain water from sump pump', 'sump pump lid gasket']"


In [143]:
query = construct_greedy_query(phrase, min_n_keywords = 26)

In [118]:
pos = query.rfind('LIMIT 1')

In [119]:
query = query[:pos] + query[pos + len('LIMIT 1'):]

In [137]:
%%time
foo = graph.run(query).to_table()

CPU times: user 0 ns, sys: 2.36 ms, total: 2.36 ms
Wall time: 442 ms


In [138]:
foo

group
"['sump pump on switch', 'turn sump pump off in winter', 'sump pump cover install', 'watchdog sump pump float switch', '20 year old sump pump', 'cost to replace sump pump in basement', 'pit boss sump pump', 'sump pump problem solving', 'sump pump preventative maintenance', 'my sump pump smells like sewage']"


In [99]:
len(foo)

8

In [100]:
len(foo[1][0])

18

In [25]:
%%time
foo = graph.run(construct_greedy_query(phrase, min_n_keywords = 25)).to_table()

CPU times: user 3.75 ms, sys: 0 ns, total: 3.75 ms
Wall time: 4.42 s


In [26]:
foo

group
"['myers battery backup sump pump manual', 'rain barrel sump pump discharge', 'diy sump pump install crawl space', 'sump pump power pack', 'sump pump cooling oil', 'what does it cost for a sump pump', 'difference between sump pump and well pump', 'zoeller m53 sump pump float adjustment', 'sump pump use basement', 'sump pump on switch', 'turn sump pump off in winter', 'small sump pump with internal float', 'why does sump pump need a vent hole', 'pit boss sump pump', 'sump pump problem solving', 'sump pump preventative maintenance', 'my sump pump smells like sewage', 'sump pump backup systems water pressure', 'sump pump cover install', 'watchdog sump pump float switch', '20 year old sump pump', 'cost to replace sump pump in basement', 'sump pump water shut off', 'sump pump window well', 'sump pump lid gasket']"
